# Backblaze Drive Stats AI Agent Demo - DeepSeek Edition

This notebook explores whether a DeepSeek LLM can be substituted for OpenAI gpt-4o-mini in a simple AI Agent implementation that answers questions about hard drive reliability based on over ten years of raw data from the Backblaze [Drive Stats](https://www.backblaze.com/drivestats) project.

Since this notebook focuses on the use of DeepSeek, it skips some explanation of the AI Agent implementation. You can find the full commentary in the original notebook at [agent_demo.ipynb](agent_demo.ipynb).

Note that, due to the non-deterministic nature of large language models (LLMs), as well as changes in the LangChain framework, and other dependencies, over time, you will likely see different results if you run this notebook yourself.

## Acknowledgements

This work is originally based on the LangChain [Build a Question/Answering system over SQL data](https://python.langchain.com/docs/tutorials/sql_qa/) tutorial.

## Install Dependencies

First, install the required Python packages. You will need to restart the Jupyter kernel before you can use newly-installed packages. You can uncomment the second line, or manually restart the kernel.

In [1]:
%pip install --upgrade --quiet -r requirements.txt

# Uncomment this line to restart the kernel so that it uses the new modules
# get_ipython().kernel.do_shutdown(restart=True)

## Debugging

If things aren't working as they should, there are a couple of ways to gain more insight. First, you can enable debug logging in the OpenAI SDK:

In [2]:
import os

# Uncomment the following line to enable debug logging in the OpenAI SDK
# os.environ["OPENAI_LOG"] = "debug"

If you need more detail, you can use a logging HTTP client to display the requests and responses exchanged with the API service:

In [3]:
from llm.utils import logging_client

# Uncomment the following line to use the logging HTTP client
#extra_kwargs["http_client"] = logging_client()

## Getting a Connection to the Database

LangChain's `SQLDatabase` class provides access to databases via the [SQLAlchemy](https://www.sqlalchemy.org/) open-source Python library. Here we're using the [Trino](https://trino.io/) open source SQL query engine to query the data set.

In [4]:
import os

from langchain_community.utilities import SQLDatabase
import warnings

# Suppress warnings triggered by SQLAlchemy not finding Trino partition columns
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    db = SQLDatabase.from_uri('trino://admin@localhost:8080/iceberg/drivestats')

Run a simple query to check that we can access data.

In [5]:
db.run("SELECT COUNT(*) FROM drivestats")

'[(564566016,)]'

Note: since the original notebook was written, we [updated the Drive Stats data set for Q4 2024](https://www.backblaze.com/blog/backblaze-drive-stats-for-2024/), adding 27,345,292 rows to the `drivestats` table.

## Using DeepSeek to Generate a SQL Query

According to the [documentation](https://api-docs.deepseek.com/), DeepSeek has an OpenAI-compatible API, so we just move the DeepSeek API key from its environment variable to the OpenAI API key environment variable and create a `ChatOpenAI` instance, supplying the DeepSeek API base URL as instructed.

DeepSeek currently supports two models: `deepseek-reasoner` (aka DeepSeek R1) and `deepseek-chat` (based on [DeepSeek-V3](https://api-docs.deepseek.com/news/news1226)). Let's try the much-talked-about DeepSeek R1:

In [6]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# The .env file needs at least DEEPSEEK_API_KEY, and may also contain
# OPENAI_API_KEY. Move the DeepSeek API key to the OpenAI environment
# variable
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.pop("DEEPSEEK_API_KEY")

# extra_kwargs allows us to easily configure debugging - see the "Debugging" section above.
# We need to set it to an empty dict here if it hasn't already been defined.
try:
    extra_kwargs
except NameError:
    extra_kwargs = {}

llm = ChatOpenAI(model="deepseek-reasoner", base_url='https://api.deepseek.com', **extra_kwargs)

As in the original notebook, we use the default LangChain SQL query system prompt as a starting point.

In [7]:
from langchain import hub

# Suppress the LangSmith API key warning - we don't need it to do hub.pull
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


A helper function populates the prompt parameters and prompts the model to generate a SQL query.

In [8]:
from typing_extensions import Annotated, TypedDict

class QueryOutput(TypedDict):
    """Generated SQL query."""
    query: Annotated[str, ..., "Syntactically valid SQL query."]


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


def write_query(state: State):
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"].rstrip(';')}

Now we can try generating a query for a simple question, as we did with OpenAI in the original notebook.

In [9]:
import traceback
import openai

question = {"question": "How many drives are there?"}

# We catch the OpenAI exception and print the stack trace, since
# we don't want to propagate it to Jupyter and halt notebook execution
try:
    query = write_query(question)
    print(query)
except openai.OpenAIError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/1870427836.py", line 9, in <module>
    query = write_query(question)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/675984923.py", line 25, in write_query
    result = structured_llm.invoke(prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 5360, in invoke
    return self.bound.invoke(
           ^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/language_models/chat_models.py", line 284,

What went wrong? Searching for the error turns up a [comment from a LangChain engineer](https://github.com/langchain-ai/langchain/issues/29282#issuecomment-2599971695) explaining that we need to use `BaseChatOpenAI` rather than `ChatOpenAI` since it '[...] accommodates many APIs that are similar to OpenAI. It uses tool calling for structured output by default'.

Redefining `llm` accordingly...

In [10]:
from dotenv import load_dotenv
from langchain_openai.chat_models.base import BaseChatOpenAI

load_dotenv()

llm = BaseChatOpenAI(model="deepseek-reasoner", base_url='https://api.deepseek.com', **extra_kwargs)

...and trying once more to generate a query:

In [11]:
question = {"question": "How many drives are there?"}

try:
    query = write_query(question)
    print(query)
except openai.OpenAIError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/1522587425.py", line 4, in <module>
    query = write_query(question)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/675984923.py", line 25, in write_query
    result = structured_llm.invoke(prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 5360, in invoke
    return self.bound.invoke(
           ^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/language_models/chat_models.py", line 284,

Another error. This time, the error message gives us some concrete advice on proceeding:

```text
The last message of deepseek-reasoner must be a user message, or an assistant message with prefix mode on (refer to https://api-docs.deepseek.com/guides/chat_prefix_completion).
```

We can replace the off-the-shelf prompt with an equivalent that ends with a user message:

In [12]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

query_prompt_template = ChatPromptTemplate([
    ("system", textwrap.dedent("""Given an input question, create a
    syntactically correct {dialect} query to run to help find the answer.
    Unless the user specifies in his question a specific number of examples
    they wish to obtain, always limit your query to at most {top_k} results.
    You can order the results by a relevant column to return the most
    interesting examples in the database.

    Never query for all the columns from a specific table, only ask for a the
    few relevant columns given the question.

    Pay attention to use only the column names that you can see in the schema
    description. Be careful to not query for columns that do not exist. Also,
    pay attention to which column is in which table.

    Only use the following tables:
    {table_info}""")),
    ("human", "Question: {input}"),
])

Now we can retry the `write_query()` call:

In [13]:
question = {"question": "How many drives are there?"}

try:
    query = write_query(question)
    print(query)
except openai.OpenAIError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/1522587425.py", line 4, in <module>
    query = write_query(question)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/675984923.py", line 25, in write_query
    result = structured_llm.invoke(prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 5360, in invoke
    return self.bound.invoke(
           ^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/language_models/chat_models.py", line 284,

Yet another error! It's becoming clear that `deepseek-reasoner` is not the correct tool for this job.

DeepSeek provides another model, `deepseek-chat`, that looks to be a better fit for this application. Does it work with the `ChatOpenAI` class?

Recreating `llm`, and using the off-the-shelf prompt we started with...

In [14]:
llm = ChatOpenAI(model="deepseek-chat", base_url='https://api.deepseek.com', **extra_kwargs)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

...and retrying the `write_query()` call:

In [15]:
question = {"question": "How many drives are there?"}

try:
    query = write_query(question)
    print(query)
except openai.OpenAIError:
    traceback.print_exc()

Traceback (most recent call last):
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/1522587425.py", line 4, in <module>
    query = write_query(question)
            ^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/ms/57rpbl_x5q91l6b9ng1kv8bm0000gq/T/ipykernel_43762/675984923.py", line 25, in write_query
    result = structured_llm.invoke(prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 3022, in invoke
    input = context.run(step.invoke, input, config, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/runnables/base.py", line 5360, in invoke
    return self.bound.invoke(
           ^^^^^^^^^^^^^^^^^^
  File "/Users/ppatterson/src/ai-agent-demo/.venv/lib/python3.12/site-packages/langchain_core/language_models/chat_models.py", line 284,

The answer is no; we'll have to use `BaseChatOpenAI`:

In [16]:
llm = BaseChatOpenAI(model="deepseek-chat", base_url='https://api.deepseek.com', **extra_kwargs)

One more time:

In [17]:
question = {"question": "How many drives are there?"}

try:
    query = write_query(question)
    print(query)
except openai.OpenAIError:
    traceback.print_exc()

{'query': 'SELECT COUNT(*) AS total_drives FROM drivestats LIMIT 10'}


At last, we have a query! However, as was the case with OpenAI, the query is valid SQL, but it won’t give us the answer we are looking for. Here's the revised question we used with OpenAI that enabled the LLM to generate the correct query:

In [18]:
question = {"question": "Each drive has its own serial number. How many drives are there?"}

query = write_query(question)

query

{'query': 'SELECT COUNT(DISTINCT serial_number) AS total_drives FROM drivestats'}

With the additional information, DeepSeek generates a similarly correct query.

Another helper function executes the query:

In [19]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

def execute_query(state: State):
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(state["query"])}

Now we can execute the generated query:

In [20]:
result = execute_query(query)

result

{'result': '[(445773,)]'}

Our last helper function uses the LLM to generate an answer from the query results:

In [21]:
def generate_answer(state: State):
    prompt = (
        "Given the following user question, corresponding SQL query, "
        "and SQL result, answer the user question.\n\n"
        f'Question: {state["question"]}\n'
        f'SQL Query: {state["query"]}\n'
        f'SQL Result: {state["result"]}'
    )
    response = llm.invoke(prompt)
    return {"answer": response.content}

As with OpenAI, we can test the `generate_answer` helper function in isolation:

In [22]:
answer = generate_answer(question | query | result)

answer['answer']

'The SQL query counts the number of distinct serial numbers in the `drivestats` table, which represents the total number of unique drives. Based on the SQL result, there are **445,773** drives.'

DeepSeek gives us a natural language response containing the correct data; at the time of writing (Feb 18, 2025), there are indeed 445,773 drives in the Drive Stats data set, 15,309 more than there were when we wrote the original notebook.

We use LangGraph to compile the steps into a single `graph` object:

In [23]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence(
    [write_query, execute_query, generate_answer]
)
graph_builder.add_edge(START, "write_query")
graph = graph_builder.compile()

Mimicking the path we followed with OpenAI, let’s _stream_ a new question through the flow so we can see the output of each step as it is executed:

In [24]:
from IPython.display import display_markdown
from more_itertools import one

# Add a horizontal rule before each step output, apart from the first
hr = ''
for step in graph.stream(
    {"question": "Each drive has its own serial number. How many drives did each data center have on 9/1/2024?"}, stream_mode="updates"
):
    # For this graph, step has the form {'node_name': {'key', 'message'}},
    # where node_name is one of 'write_query', 'execute_query', or
    # 'generate_answer' and key is one of 'query', 'result' or 'answer'.
    #
    # one gives us the first element of an iterable, allowing us to
    # conveniently get the wrapped value.
    state = one(step.values())
    message = hr + one(state.values())
    if len(hr) == 0:
        hr = '---\n'
    display_markdown(message, raw=True)

SELECT datacenter, COUNT(DISTINCT serial_number) AS drive_count FROM drivestats WHERE date = DATE '2024-09-01' GROUP BY datacenter LIMIT 10

---
[('phx1', 89477), ('', 24080), ('ams5', 16139), ('sac0', 78444), ('iad1', 22800), ('sac2', 60775)]

---
The SQL query results show the number of drives (based on distinct serial numbers) in each data center on September 1, 2024. Here's the breakdown:

- **phx1**: 89,477 drives  
- **ams5**: 16,139 drives  
- **sac0**: 78,444 drives  
- **iad1**: 22,800 drives  
- **sac2**: 60,775 drives  
- **Unspecified data center (empty string)**: 24,080 drives  

These are the top 10 results returned by the query, and it appears that one entry has an unspecified or missing data center identifier.

Interestingly, where OpenAI generated a query that incorrectly compared the `date` column to a string value, DeepSeek generated the correct SQL query for Trino, avoiding OpenAI's error by including the `DATE` type identifier.

## Creating a ReAct AI Agent with LangGraph

The LangGraph framework gives you the capability to create AI agents based on arbitrarily complex logic. In this article, I’ve used its prebuilt [ReAct](https://react-lm.github.io/) (Reason+Act) agent, since it neatly demonstrates the agent concept, rewriting the SQL query repeatedly in response to database errors.

There are three steps to creating the agent. The first is to create an instance of LangChain's [SQLDatabaseToolkit](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html), passing it the database and model, and obtain its list of tools:

In [25]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x107c1b2f0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x107c1b2f0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x107c1b2f0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

The `tools` list contains tools that execute queries, retrieve the names, schemas and content of database tables, and check SQL query syntax.

The next step is to retrieve a suitable prompt template from the Prompt Hub and populate the template placeholders:

In [26]:
from langchain import hub

# Suppress the LandSmith API key warning - we don't need it to do hub.pull
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].pretty_print()

================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

In [27]:
system_message = prompt_template.format(dialect=db.dialect, top_k=10)

Now we can create an instance of the prebuilt agent:

In [28]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

Again, we can stream the agent’s execution, using the same question as before, to show us each step of its operation.

In [29]:
for step in agent_executor.stream(
    {"messages": [{"role": "user", "content": "Each drive has its own serial number. How many drives did each data center have on 9/1/2024?"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Each drive has its own serial number. How many drives did each data center have on 9/1/2024?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0_9a1e5658-80be-41b0-90b4-e5295bf75b3d)
 Call ID: call_0_9a1e5658-80be-41b0-90b4-e5295bf75b3d
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

drivestats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0_c67ca9eb-ca81-45c7-984d-7bda59906aea)
 Call ID: call_0_c67ca9eb-ca81-45c7-984d-7bda59906aea
  Args:
    table_names: drivestats
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE drivestats (
	date DATE, 
	serial_number VARCHAR, 
	model VARCHAR, 
	capacity_bytes BIGINT, 
	failure INTEGE

The answer from the DeepSeek agent is consistent with both the answer from the earlier graph in this notebook and the OpenAI agent.

Now we'll see if DeepSeek can calculate the annualized failure rate (AFR) of a drive model. As we did with OpenAI, we’ll use the Seagate ST4000DM000.

In [31]:
for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": "Each drive has its own serial number. What is the annualized failure rate of the ST4000DM000 drive model?"}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Each drive has its own serial number. What is the annualized failure rate of the ST4000DM000 drive model?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0_b09f11ba-33a1-45f3-881e-1763b45d332c)
 Call ID: call_0_b09f11ba-33a1-45f3-881e-1763b45d332c
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

drivestats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0_83f32b68-b2c9-4c25-9c3b-e016459361c5)
 Call ID: call_0_83f32b68-b2c9-4c25-9c3b-e016459361c5
  Args:
    table_names: drivestats
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE drivestats (
	date DATE, 
	serial_number VARCHAR, 
	model VARCHAR, 
	capacity_bytes BIGINT, 
	failure INTEGER, 

As was the case with OpenAI, the DeepSeek agent shows its working, but, again, arrives at the wrong answer. This time, it simply calculated the overall failure rate of ST4000DM000 drives, ignoring the instruction to provide the _annualized_ failure rate.

Continuing to follow the path we established in the previous notebook, we'll provide a prompt with detailed instructions for calculating AFR:

In [32]:
prompt_template.messages[0].prompt.template += """
Each row of the drivestats table records one day of a drive’s operation, and contains the serial number of a drive, its model name, capacity in bytes, whether it failed on that day, SMART attributes and identifiers for the slot, pod, vault, cluster and data center in which it is located.

Use this calculation for the annualized failure rate (AFR) for a drive model over a given time period:

1. **drive_days** is the number of rows for that model during the time period.
2. **failures** is the number of rows for that model during the time period where **failure** is equal to 1.
3. **annual failure rate** is 100 * (**failures** / (**drive_days** / 365)).

Use double precision arithmetic in the calculation to avoid truncation errors. To convert an integer **i** to a double, use CAST(**i** AS DOUBLE)

Note that the date column is a DATE type, not a string. Use the DATE type identifier when comparing the date column to a string.

Do not add a semi-colon suffix to SQL queries."""

We need to recreate the `system_message` and `agent_executor` with the new prompt template:

In [33]:
system_message = prompt_template.format(dialect=db.dialect, top_k=10)
agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

Now we can repeat our request for the ST4000DM000 AFR:

In [34]:
for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": "What is the annual failure rate of the ST4000DM000 drive model?"}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the annual failure rate of the ST4000DM000 drive model?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0_e7e0c4b3-5178-4d11-83bd-a470c66120a6)
 Call ID: call_0_e7e0c4b3-5178-4d11-83bd-a470c66120a6
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

drivestats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0_40807e1a-b592-4c46-8df4-6da0f6c497eb)
 Call ID: call_0_40807e1a-b592-4c46-8df4-6da0f6c497eb
  Args:
    table_names: drivestats
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE drivestats (
	date DATE, 
	serial_number VARCHAR, 
	model VARCHAR, 
	capacity_bytes BIGINT, 
	failure INTEGER, 
	smart_1_raw BIGINT, 
	sm

The DeepSeek-powered agent comes up with the correct number for the ST4000DM000 AFR, 2.63%, but, for some reason, its response is simply the result set it received from querying the Trino SQL engine, rather than a natural language statement.

As we did with OpenAI, let’s ask the DeepSeek agent for a statistic that we can corroborate from the [Backblaze Drive Stats for Q3 2024](https://www.backblaze.com/blog/backblaze-drive-stats-for-q3-2024/) blog post.

In [38]:
for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": "What was the annual failure rate of the ST8000NM000A drive model in Q3 2024?"}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What was the annual failure rate of the ST8000NM000A drive model in Q3 2024?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0_cbe35112-a7af-40cb-b8d4-49e0777623d6)
 Call ID: call_0_cbe35112-a7af-40cb-b8d4-49e0777623d6
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

drivestats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0_86296fc6-7d4e-4f19-818c-9a9bee489072)
 Call ID: call_0_86296fc6-7d4e-4f19-818c-9a9bee489072
  Args:
    table_names: drivestats
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE drivestats (
	date DATE, 
	serial_number VARCHAR, 
	model VARCHAR, 
	capacity_bytes BIGINT, 
	failure INTEGER, 
	smart_1_raw BIGINT, 
	smart

Again, DeepSeek provides a "raw" response. The figure, 1.61%, matches that given in the Drive Stats blog post.

Finally, let’s ask a more convoluted question, including the constraints given in the blog post:

In [43]:
for step in agent_executor.stream(
        {"messages": [{"role": "user", "content": "Considering only drive models which had at least 100 drives in service at the end of the quarter and which accumulated 10,000 or more drive days during the quarter, which drive had the most failures in Q3 2024, and what was its failure rate?"}]},
        stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Considering only drive models which had at least 100 drives in service at the end of the quarter and which accumulated 10,000 or more drive days during the quarter, which drive had the most failures in Q3 2024, and what was its failure rate?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_0_9bc4aa2b-00ec-4a15-8b5c-243d6ab7c8e0)
 Call ID: call_0_9bc4aa2b-00ec-4a15-8b5c-243d6ab7c8e0
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

drivestats
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_0_f15c39c5-92ec-4fc6-9b14-3b25565c582f)
 Call ID: call_0_f15c39c5-92ec-4fc6-9b14-3b25565c582f
  Args:
    table_names: drivestats
================================= Tool Message =================================
Name: sql_

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. (insufficient tool messages following tool_calls message)", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}

This time, the agent generates an invalid SQL query, and the agent invocation subsequently ends with an error message.